In [1]:
import sys
import os
import json
import pandas as pd

sys.path.append('./tools/experiment')
sys.path.append('./tools/client')

import prometheus
import grafana
import experiment

data_root = "/home/ict/appProfile/data"
grafana_url_base = "http://localhost:13001/d/aio/allinone?orgId=1"

In [14]:
class Workload:
    descriptor = ""
    start_time = 0
    end_time = 0
    url = ""
    log = ""
    execute = {}
    addtion = {}

    def __init__(self, descriptor="", start_time=0, end_time=0, url="", log="", execute={}, addtion={}):
        self.descriptor = descriptor
        self.start_time = start_time
        self.end_time = end_time
        self.log = log
        self.execute = execute
        self.addtion = addtion

    def to_dict(self):
        return {
            "descriptor": self.descriptor,
            "start_time": self.start_time,
            "end_time": self.end_time,
            "url": self.url,
            "log": self.log,
            "execute": self.execute,
            "addtion": self.addtion
        }

In [15]:
exp = experiment.Experiment(
    mode="standard",
    start_time = 202309250000,
    end_time = 202309251111,
).with_mem_stress(
    memrate_range = range(1, 2),
)

# Workload/Benchmark info from benchmark log
start = 1695597616000
end = 1695608266000


workload = Workload(**{"descriptor": "redis", "start_time": start, "end_time": end, "url": f"{grafana_url_base}&from={start}&to={end}", "log": ""})
info_time_ranges = {"redis": [workload]}

# Client Bench Metric info from bench result
qos_info = {"qps": 10000, "p95_latency": 113}
qos_infos = [qos_info]


# Stress info from experiment setting
for k, v in info_time_ranges.items():
    if len(exp.stress_info_per_epoch) != 0:
        for i, e in enumerate(info_time_ranges[k]):
            e["addition"] = {
                "metric": qos_infos[i],
                "stress": exp.stress_info_per_epoch[i]
            }

info_time_ranges

TypeError: 'Workload' object does not support item assignment

In [3]:
grafana_auth = "admin:admin@"
grafana_server = "10.208.130.243:3001"
prom_server = "10.208.130.243:9090"
aio_db = "aio"
step = "4s"

pclient = prometheus.client(prom_server)
gclient = grafana.client(grafana_server, grafana_auth)
targets = gclient.get_db(aio_db)["panels"][0]["targets"]

dfs = [pclient.targets_to_df(targets, info["start_time"] / 1000, info["end_time"] / 1000, step) for info in v for k, v in info_time_ranges.items()]

In [4]:
combined = pd.concat(dfs).fillna(0).sort_values(by=['Time'])
combined = combined.reset_index()
combined["Time"] = combined["Time"] * 1000

In [6]:
# save to data dir
dir_path = os.path.join(data_root, exp.dir_name())
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
combined.to_csv(os.path.join(dir_path, "merged.csv"))

with open(os.path.join(dir_path, "exp.json"), 'w') as f:
    json.dump(exp.to_dict(), f)

with open(os.path.join(dir_path, "date.json"), 'w') as f:
    json.dump(info_time_ranges, f)